In [22]:
# 使用SimpleTagBased算法对Delicious2K数据进行推荐
# 原始数据集：https://grouplens.org/datasets/hetrec-2011/
# 数据格式：userID     bookmarkID     tagID     timestamp
import random
import math
import operator
import pandas as pd
import time

start=time.process_time()
file_path = "./user_taggedbookmarks-timestamps.dat"
# 字典类型，保存了user对item的tag，即{userid: {item1:[tag1, tag2], ...}}
records = {}
# 训练集，测试集
train_data = dict()
test_data = dict()
# 用户标签，商品标签
user_tags = dict()
tag_items = dict()
user_items = dict()
tags_users=dict()

# 使用测试集，计算准确率和召回率
def precisionAndRecall(method,N):
    hit = 0
    h_recall = 0
    h_precision = 0
    for user,items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = method(user, N)
        for item,rui in rank:
            if item in items:
                hit = hit + 1
        h_recall = h_recall + len(items)
        h_precision = h_precision + N
    #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
    # 返回准确率 和 召回率
    return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))

def simpleRecommend(user, N):  #SimpleTagBased
    recommend_items=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    tagged_items = user_items[user]     
    for tag, wut in user_tags[user].items():
        #print(self.user_tags[user].items())
        for item, wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            #print('wut = %s, wti = %s' %(wut, wti))
            if item not in recommend_items:
                recommend_items[item] = wut * wti
            else:
                recommend_items[item] = recommend_items[item] + wut * wti
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

def normRecommend(user, N):
    def total_wut():
        totalSum = 0
        for i in user_tags[user].values():
            totalSum+=i
        return totalSum
    def totalWti():
        totalSum=0
        for i in tag_items[tag].values():
            totalSum+=i
        return totalSum
    
    recommend_items=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    total_wut=total_wut()
    tagged_items = user_items[user]  #对于该user的所有item使用次数的数据   
    for tag, wut in user_tags[user].items():  #对于该user下不同tag的使用次数的数据
        #print(self.user_tags[user].items())
        total_wti=totalWti()
        for item, wti in tag_items[tag].items(): #对于该tag被用于不同item的次数
            if item in tagged_items: 
                continue
            #print('wut = %s, wti = %s' %(wut, wti))
            if item not in recommend_items: #相同的技巧，填充空字典
                recommend_items[item] = (wut/total_wut) * (wti/total_wti)
                #recommend_items[item] = (wut/total_wut) * (wti)
            else:
                recommend_items[item] = recommend_items[item] + (wut/total_wut) * (wti/total_wti)
                #recommend_items[item] = recommend_items[item] + (wut/total_wut) * (wti)
    # 把recommend_items的键和值提成元组，然后按照值（分数）从高到低排，只取N个
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

def TFIDF_Recommend(user, N):
    def total_tags_users():
        totalSum=0
        for i in tags_users[tag].values():
            totalSum+=i
        return totalSum
    recommend_items=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    tagged_items = user_items[user]  #对于该user的所有item使用次数的数据   
    for tag, wut in user_tags[user].items():  #对于该user下不同tag的使用次数的数据
        #print(self.user_tags[user].items())
        total_tags_user=total_tags_users()
        for item, wti in tag_items[tag].items(): #对于该tag被用于不同item的次数
            if item in tagged_items: 
                continue
            #print('wut = %s, wti = %s' %(wut, wti))
            if item not in recommend_items: #相同的技巧，填充空字典
                recommend_items[item] = (wut / math.log(1+total_tags_user)) * wti
                
            else:
                recommend_items[item] = recommend_items[item] + (wut / math.log(1+total_tags_user)) * wti
                #recommend_items[item] = recommend_items[item] + (wut/total_wut) * (wti)
    # 把recommend_items的键和值提成元组，然后按照值（分数）从高到低排，只取N个
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]
    
    


# 使用测试集，对推荐结果进行评估
def testRecommend():
    print("推荐结果评估")
    print("%3s %23s %23s %24s" % ('',"SimpleTagBased算法",'NormTagBased算法','TagBased-TFIDF算法'))
    print("  --------------------------------------------------------------------------------")
    print("%3s %10s %10s %10s %10s %10s %10s" % ('N',"精确率",'召回率',"精确率",'召回率',"精确率",'召回率'))
    for n in [5,10,20,40,60,80,100]:
        s_precision,s_recall = precisionAndRecall(simpleRecommend,n)
        n_precision,n_recall = precisionAndRecall(normRecommend,n)
        t_precision,t_recall = precisionAndRecall(TFIDF_Recommend,n)
        print("%3d %12.3f%% %11.3f%% %11.3f%% %11.3f%% %11.3f%% %11.3f%%" %  \
              (n, s_precision * 100, s_recall * 100, n_precision * 100, n_recall * 100, t_precision * 100, t_recall * 100))
        
# 数据加载
def load_data():
    print("开始数据加载...")
    df = pd.read_csv(file_path, sep='\t')
    for i in range(len(df)):
        uid = df['userID'][i]
        iid = df['bookmarkID'][i] #物品id
        tag = df['tagID'][i]
        # 键不存在时，设置默认值{}
        records.setdefault(uid,{})  #键为userID，值为空字典
        records[uid].setdefault(iid,[]) # 把键uid的值（字典），添加键iid，值为空列表list
        records[uid][iid].append(tag) # 给uid键下的值（字典）中键为idd的值（列表），赋值为tag
    print("数据集大小为 %d." % (len(df)))
    print("设置tag的人数 %d." % (len(records)))
    print("数据加载完成\n")
    
    ##
    print("df 的前五行是")
    #print(df)
    print(df.head())
    print("")
    print("df 的10到20行，前3列是")
    print(df.iloc[10:20,:3])
    print("")
    print("df 的后五行是")
    print(df.tail())

# 将数据集拆分为训练集和测试集
# 字典类型，保存了user对item的tag，即{userid: {item1:[tag1, tag2], ...}}
def train_test_split(ratio, seed=100):
    random.seed(seed)
    for u in records.keys():  # 第一循环，总共437593个
        for i in records[u].keys():  #第二循环，数量为item的数量
            # ratio比例设置为测试集
            if random.random()<ratio: #小于ratio为训练集
                test_data.setdefault(u,{}) #把userID 对应的值添加为空字典
                test_data[u].setdefault(i,[]) #把userID 对于的空字典的 键变为item ID, 值为空列表
                for t in records[u][i]: # 把userID 对应的 itemID 的值（列表）添加tag进去
                    test_data[u][i].append(t)
            else: #同理，把大于ratio的随机数对应的userID产生训练集
                train_data.setdefault(u,{})
                train_data[u].setdefault(i,[])
                for t in records[u][i]:
                    train_data[u][i].append(t)        
    print("训练集样本数 %d, 测试集样本数 %d" % (len(train_data),len(test_data)))
    
# 设置矩阵 mat[index, item] = 1
def addValueToMat(mat, index, item, value=1):
    if index not in mat:  #index不在mat里
        mat.setdefault(index,{})  #把index作为新键，值为空字典
        mat[index].setdefault(item,value) # 把空字典的键设为item，值为1
    else: #如果index在mat里
        if item not in mat[index]: # 如果item不在index（猜测index为userID）对应的值里
            mat[index][item] = value # 把该物品的标记设为1 ，这个妙在为把各种值填入了空的字典 
        else:
            mat[index][item] += value # 如果该用户已经用过该item， 把该item的数量增加1



# 使用训练集，初始化user_tags, tag_items, user_items
def initStat():
    records=train_data
    for u,items in records.items(): #对每一个userID 和对应字典（item）进行循环
        for i,tags in items.items(): # 把每一个itemID和tag的list 进行循环
            for tag in tags: #对tag list 进行循环
                #print tag
                # 用户和tag的关系
                addValueToMat(user_tags, u, tag, 1)
                # tag和item的关系
                addValueToMat(tag_items, tag, i, 1)
                # 用户和item的关系
                addValueToMat(user_items, u, i, 1)
                # tag和用户的关系
                addValueToMat(tags_users, tag, u, 1)
    print("user_tags, tag_items, user_items, tags_users初始化完成.")
    print("user_tags 大小 %d, tag_items 大小 %d, user_items 大小 %d, tags_users 大小 %d"% \
          (len(user_tags), len(tag_items), len(user_items), len(tags_users)))

# 数据加载
load_data()
# 训练集，测试集拆分，20%测试集
train_test_split(0.2)
initStat()
testRecommend()
elapsed = (time.process_time()-start)
print("")
print("运行时间: ", elapsed)


开始数据加载...
数据集大小为 437593.
设置tag的人数 1867.
数据加载完成

df 的前五行是
   userID  bookmarkID  tagID      timestamp
0       8           1      1  1289255362000
1       8           2      1  1289255159000
2       8           7      1  1289238901000
3       8           7      6  1289238901000
4       8           7      7  1289238901000

df 的10到20行，前3列是
    userID  bookmarkID  tagID
10       8          10      1
11       8          10     11
12       8          11      1
13       8          11     12
14       8          11     13
15       8          14      1
16       8          14     15
17       8          14     16
18       8          14     17
19       8          14     18

df 的后五行是
        userID  bookmarkID  tagID      timestamp
437588  108035       30993    193  1277495315000
437589  108035       30993    673  1277495315000
437590  108035       30994    130  1277223715000
437591  108035       30994    267  1277223715000
437592  108035       30994   4943  1277223715000
训练集样本数 1860, 测试集样本数 1793
use